# Facility Design Project

<center>ALMOST FINAL - 26.12.2020</center>

# 0. Imports

In [959]:
import pandas as pd
import numpy as np
import pickle as p
import random
# pip install dataframe-image
#import dataframe_image

random.seed(666)

## 1. Define Classes

## 1.0. Exceptions

In [960]:
class CrashError(Exception):
    # If the target cell has already robot in there --> a crash has happened!
    pass

class RobotWithEscortError(Exception):
    # If the robot is taking an item, and there is no such thing next to it in the cell
    pass

class ItemCrashError(Exception):
    # if there is already an item in the destination cell; it should be empty when applying the step.
    pass

class ItemError(Exception):
    # The item that the robot needs to apply three/five steps on is not near the robot
    pass

class LocationError(Exception):
    # The item that the robot needs to apply three/five steps on is not near the robot
    pass

### 1.1. Item

In [961]:
class Item:
    def __init__(self,item_number,exit=0,side=0):
        self.number = item_number # catalog number
        self.exit = exit          # Can be either False/True. True means that the item needs to be taken out of the warehouse
        self.side = side          # 1 if the item is located on the left side of the warehouse, 2 on the right side, 3 if exactly on the center
    
    def takenout(self):
        self.number = 999         # only when the item were taken out at the I/O location
    
    def contains(self):
        return True # it is an item
    
    def __str__(self):
        if self.exit:
            return str("%s to exit"%(self.number))
        else:
            return str("%s"%(self.number))

### 1.2. Escort

In [962]:
class Escort:
    def __init__(self, robot_id):
        self.robot_id = robot_id # this escort belong to this robot_id
        self.number = 0          # escort number
        self.exit = 0
    
    def contains(self):
        return False # it isn't an item but an escort
    
    def __str__(self):
        return str("0")

### 1.3. Robot

In [963]:
class Robot:
    def __init__(self,robot_id,side):
        self.id = robot_id
        self.item_to_take = None     # will be equal to the item intended to be taken out
        self.currently_taking = None # will be equal to the item's number the robot is taking
        self.path = None             # the steps the robot will take
        self.side = None             # The robot can exit items only on the relevant side of the warehouse
    
    def robot_will_take(self, item_number):
        self.item_to_take = item_number
        self.currently_taking = None
    
    def robot_is_taking(self, item_number):
        self.item_to_take = None
        self.currently_taking = item_number
    
    def reset(self):
        self.item_to_take = None
        self.currently_taking = None
        self.path = None
        
    def is_free(self):
        if self.item_to_take == None and self.currently_taking == None and self.path == None:
            print("robot %s is free"%(self.id))
            return True
        else:
            print("robot %s is not free"%(self.id))
            return False
    
    def __str__(self):
        return str("Robot %s"%(self.id))

### 1.4. Cell

In [964]:
class Cell:
    def __init__(self, item_number=0, exit=0, robot_id=0, side=0): 
        # A cell can either contain: (item,robot) (escort,robot) (escort,) (item,)
        if item_number == 0: #is it an escort?
            self.item = Escort(robot_id)                   # Escort in cell
            self.robot = Robot(robot_id,side)              # We assume that the robot, at the beginning, is in the place of the escorts.
        else:
            self.item = Item(item_number, exit, side)      # item in cell, with respect to his location in the warehouse
            self.robot = ''
            
    def assignRobot(self, robot):
        self.robot = robot
        
    def assignItem(self, item):
        self.item = item
            
    def __repr__(self):
        rbt = self.robot
        itm = self.item
        return "[%s,%s]"%(itm,rbt)
        
    def __str__(self):
        rbt = self.robot
        itm = self.item
        return "[%s,%s]"%(itm,rbt)

### 1.5. Warehouse

In [965]:
class Warehouse:
    
    #########################################################################################################################
    ############### WAREHOUSE INTIALIZATION
    #########################################################################################################################
    
    def __init__(self, warehouse_filename, items_to_exit_filename):
        
        self.distances_left = []               # initiate list of distances of items from the left of the I/O
        self.distances_right = []              # initiate list of distances of items from the right of the I/O
        self.robot_side = {}                   # dictionary with pairs of (robot_id: side)
        self.robot_positions = None            # will be updated every time progresses
        self.items_to_exit_positions = None    # will be updated every time progresses
        self.robots_moves = {1: [], 2: [], 3: [], 4: [], 5:[]} # history of the robot movements
        self.exited_items = {}                 # will be dictionary where the key is the item number, and the value will be the time
                                               # of exit
        self.robot_final_positions_left = [(7,0),(3,0),(0,4)]
        self.robot_final_positions_right = [(8,14),(0,14),(0,14)]
        
        # read the pickle files
        # Warehouse
        with open(warehouse_filename,'rb') as infile:
            initiated_warehouse = p.load(infile) # automatically closes the connection with the file
        
        # Items to exit
        with open(items_to_exit_filename,'rb') as infile:
            items_to_exit = p.load(infile)
            # Dictionary of items to exit with their occupation status - the key is the item number, the value is the robot id
            self.items_to_exit = {items_to_exit[i]: False for i in range(len(items_to_exit))}
        
        rows, columns = len(initiated_warehouse), len(initiated_warehouse[0])
        
        # Initiate an array and a counter of robots
        self.warehouse = pd.DataFrame(np.zeros((rows,columns))) # initiate an array
        robot_id=1 # initiate a count of robots' id
        
        robot_side = [1,2,1,2,1]
        k = 0
        
        # Fill the warehouse
        for i in range(self.warehouse.shape[0]): # fill the warehouse with cells
            for j in range(self.warehouse.shape[1]):
                number = initiated_warehouse[i][j]
                if number == 0: # it is an escort
                    # choose randomly to which side of the warehouse the robot will be
                    side = robot_side[k]
                    self.warehouse.iloc[i,j] = Cell(robot_id=robot_id, side=side)
                    self.robot_side[robot_id] = side # save the belonging of the robot
                    robot_id+=1
                    k+=1
                else:
                    to_exit = number in items_to_exit # TRUE if the item needs to be exited; FALSE otherwise
                    if j<7:
                        side=1 # located on the left of the warehouse
                    elif j>7:
                        side=2 # located on the right of the warehouse
                    else:
                        side=3 # located exactly in the middle of the warehouse
                    self.warehouse.iloc[i,j] = Cell(item_number=number,exit=to_exit,side=side)
            
        self.calculate_positions()                # Calculate all robots & items to exit positions in the warehouse
        self.calculate_distance_from_IO()         # calculate distance of the items from the I/O point
        
    
    #########################################################################################################################
    ############### ASSIGNMENTS FUNCTIONS
    #########################################################################################################################
    
    def define_robot_path(self, robot_id, steps, overwrite=True):
        #### Assign planned path to a specific robot. 
        #### If overwrite=true, overwrite the path list of the robot with the received steps.
        
        robot_loc = self.robot_positions[robot_id-1][0] # retrieve current robot's position

        if overwrite: # should we delete the current robot's route with a new one?
            self.warehouse.iloc[robot_loc].robot.path = steps
            print("Overwritten path for robot %s"%(robot_id))
            
        else: # add the planned path to the current robot's planned steps
            self.warehouse.iloc[robot_loc].robot.path.insert(0,steps)
    
    def exit_item(self, robot_id, current_time):
        #### When the item has arrived to the I/O point, remove the item from the list of items to exit,
        #### assign item "999" to the relevant cell and add the item along with the robot_id who took it.
        
        if robot_id: # if it was exited by a robot
            robot_loc = self.robot_positions[robot_id-1][0] # retrieve current robot's position
            #self.warehouse.iloc[robot_loc].robot.currently_taking = None
        
        item_number = self.warehouse.iloc[(0,7)].item.number
        self.exited_items[item_number] = current_time+1 # it takes another time unit to exit the item
        del self.items_to_exit[item_number]
        self.warehouse.iloc[(0,7)].assignItem(Item(999))
        print("Items remaining: %s"%(len(self.items_to_exit)))
        

    #########################################################################################################################
    ############### POSITIONS
    #########################################################################################################################
    
    def calculate_positions(self):
        # robots positions
        criterion = [self.warehouse.T[i].map(lambda x: x.robot != '') for i in range(self.warehouse.shape[0])]
        positions = [(i,j) for i,j in np.argwhere(criterion)] # retrieve robots positions
        # retrieve robot positions with their ID:
        current_robot_positions = [(positions[i],
                                    self.warehouse.iloc[positions[i]].robot.id) for i in range(len(positions))]
        current_robot_positions = sorted(current_robot_positions,
                                         key=lambda x: x[1]) # preserve the robots order: 1-5
        self.robot_positions = current_robot_positions # each object is with the format: ((x,y),robot_id)
        print(self.robot_positions)
        
        # items to exit positions
        criterion = [self.warehouse.T[i].map(lambda x: x.item != 0 and x.item.exit)
                     for i in range(self.warehouse.shape[0])]
        
        positions = [(i,j) for i,j in np.argwhere(criterion)] # retrieve items to exit positions
        
        self.items_to_exit_positions = positions # each object is with the format: (x,y)
        
    def calculate_distance_from_IO(self):
        # distance calculation: {(x,y) - (0,7)} where x,y is the position of the item, (0,7) the position of the I/O
        # the warehouse is splitted to two parts: column <=7 and >7
        
        self.distances_left,self.distances_right = [],[]
        
        for item in self.items_to_exit: # for items that we need to exit
            if not self.items_to_exit[item] and item!=None: # it means that the item needs to be exited
                position = self.find_item_location(item)
                distance = position[0]+abs(position[1]-7)
                item_number = self.warehouse.iloc[position].item.number
                if position[1] > 7:
                    self.distances_right.append((item_number,distance)) # pairs of: (item number,distance from I/O)
                else:
                    self.distances_left.append((item_number,distance)) # pairs of: (item number,distance from I/O)
        
        # sort items by minimum to maximum distance
        self.distances_left = sorted(self.distances_left, key=lambda x: x[1]) # sorted by distances
        self.distances_right = sorted(self.distances_right, key=lambda x: x[1]) # sorted by distances
    
    #########################################################################################################################
    ############### SEARCH & DISTANCE CALCULATIONS
    #########################################################################################################################
    
    def find_closest_robot(self, position):
        # by a given item, calculate the closest robots to it and return the relevant distances array
        # the distance will be calculated as a sum of x+y positions
        # the distances array will contain the following objects: (abs distance,robot_id)
        distances = []
        for robot in self.robot_positions:
            distance = abs(position[0]-robot[0][0])+abs(position[1]-robot[0][1])
            distances.append((distance,robot[1])) # a pair of the distance of the item from the robot and the robot's id
        return sorted(distances,key=lambda x: x[0])
    
    
    def find_item_location(self, item_number):
        # retrieve item_number position with the format (x,y)
        criterion = [self.warehouse.T[i].map(lambda x: x.item != 0 and x.item.number==item_number)
                     for i in range(self.warehouse.shape[0])]
        positions = [(i,j) for i,j in np.argwhere(criterion)] # retrieve item location
        return positions[0]
          
    #########################################################################################################################
    ############### MANHATTAN JOURNEY PLANNER
    ######################################################################################################################### 
    
    def manhattan_journey_to_item(self, robot_id, item, overwrite=True, final=False, final_loc=0):
        # calculate the path towards a location: (x,y) format, assuming the robot starts the route when it's on an escort
        
        steps = []
        
        ### Locations
        current_loc = self.robot_positions[robot_id-1][0] # retrieve current robot's position
        robot_loc = current_loc
        target_loc = self.find_item_location(item)
        print('Manhattan journey - item %s is in %s, target location for robot %s: %s'%(item,target_loc,robot_id,target_loc))
        
        ### Make sure we are with the escort
        escort_loc = self.return_to_escort(current_loc,robot_id)
        print("escort_loc: %s"%str(escort_loc))
        if escort_loc: # we are checking that the escort is with the item
            steps.extend([(current_loc,escort_loc,False)]) # one step to the escort
            current_loc = escort_loc
        
        ## Scenarios:
        ### 1. The target item's location is on the same row as the robot's
        ### 2. The target item's location is on the same column as the robot's 
        
        if target_loc[0] == current_loc[0] or target_loc[0]<=1: # target is in the same row: apply columns steps first, then rows
            decision = True
        elif target_loc[1] == current_loc[1]: # target is in the same column: apply rows first, then columns
            decision = False
        else: # choose randomly
            decision = random.choice([True, False])
            
        if final: # is this the last route for the robot?
            decision = random.choice([True, False])
            current_loc = final_loc

        if decision: 
            if target_loc[0] == 0: # make sure we are next to the item
                if 0 < target_loc[1] < 14:
                    target_loc = (target_loc[0],target_loc[1]-1)
                elif target_loc[1] == 0:
                    target_loc = (target_loc[0],1)
                elif target_loc[1] == 14:
                    target_loc = (target_loc[0],13)
                    
            columns_steps = self.columns_steps(current_loc, target_loc[1])
            middle_loc = (current_loc[0],target_loc[1]) # we have arrived to the y target
            rows_steps = self.rows_steps(middle_loc,target_loc[0])
            
            steps.extend(columns_steps)
            steps.extend(rows_steps)

        else: 
            if target_loc[0] > 1:
                target_loc = (target_loc[0]-1,target_loc[1]) # one row beneath the item
                
            rows_steps = self.rows_steps(current_loc,target_loc[0])
            middle_loc = (target_loc[0],current_loc[1]) # we have arrived to the x target
            columns_steps = self.columns_steps(middle_loc, target_loc[1])
            
            steps.extend(rows_steps)
            steps.extend(columns_steps)
        
        if steps:
            if overwrite:
                self.define_robot_path(robot_id, steps, overwrite)  # assign the relevant final planned steps
                print(steps)
            else: # in case we want to add steps to the current robot's path
                for step in reversed(steps): # adding the manhattan step like adding elements to a stack
                    self.define_robot_path(robot_id, step, overwrite)
        
        else: # we haven't planned any steps
            self.three_step(robot_id)
            return True
            
            
        if not final:
            self.warehouse.iloc[robot_loc].robot.item_to_take = item # assign the intended item to take
            print("Robot %s will take %s out"%(robot_id,self.warehouse.iloc[robot_loc].robot.item_to_take))
        else:
            print("Robot %s finished his job"%(robot_id))
        
    #########################################################################################################################
    ############### MOVEMENTS IN THE WAREHOUSE
    #########################################################################################################################
    
    def rows_steps(self, current_loc, target):
        first_movement, visited, steps = True, False, []

        if target-current_loc[0] > 0: # walking from down to up
            for x in range(current_loc[0],target,1):
                visited=True
                if first_movement:
                    step1 = [((x,current_loc[1]), (x+1,current_loc[1]), False)]
                    step2 = [((x+1,current_loc[1]), (x,current_loc[1]), True)]
                    steps.extend(step1+step2)
                    first_movement=False
                else:
                    step1 = [((x-1,current_loc[1]), (x,current_loc[1]), False)]
                    step2 = [((x,current_loc[1]), (x+1,current_loc[1]), False)]
                    step3 = [((x+1,current_loc[1]), (x,current_loc[1]), True)]
                    steps.extend(step1+step2+step3)
            # last move to the escort, but make sure we have done earlier steps!
            if visited:
                steps.extend([((target-1,current_loc[1]), (target,current_loc[1]), False)])
        else: # walking from up to down
            for x in range(current_loc[0],target,-1):
                visited=True
                if first_movement:
                    step1 = [((x,current_loc[1]), (x-1,current_loc[1]), False)]
                    step2 = [((x-1,current_loc[1]), (x,current_loc[1]), True)]
                    steps.extend(step1+step2)
                    first_movement=False
                else:
                    step1 = [((x+1,current_loc[1]), (x,current_loc[1]), False)]
                    step2 = [((x,current_loc[1]), (x-1,current_loc[1]), False)]
                    step3 = [((x-1,current_loc[1]), (x,current_loc[1]), True)]
                    steps.extend(step1+step2+step3)
            # last move to the escort, but make sure we have done earlier steps!
            if visited:
                steps.extend([((target+1,current_loc[1]), (target,current_loc[1]), False)])

        return steps

    def columns_steps(self, current_loc, y_target):
        first_movement, visited, steps = True, False, []
        
        if y_target-current_loc[1] > 0: # walking from left to right
            for y in range(current_loc[1],y_target,1):
                visited=True
                if first_movement:
                    step1 = [((current_loc[0],y), (current_loc[0],y+1), False)]
                    step2 = [((current_loc[0],y+1), (current_loc[0],y), True)]
                    steps.extend(step1+step2)
                    first_movement=False
                else:
                    step1 = [((current_loc[0],y-1), (current_loc[0],y), False)]
                    step2 = [((current_loc[0],y), (current_loc[0],y+1), False)]
                    step3 = [((current_loc[0],y+1), (current_loc[0],y), True)]
                    steps.extend(step1+step2+step3)
            # last move to the escort, but make sure we have done earlier steps!
            if visited:
                steps.extend([((current_loc[0],y_target-1), (current_loc[0],y_target), False)])
        else: # walking from right to left
            for y in range(current_loc[1],y_target,-1):
                visited=True
                if first_movement:
                    step1 = [((current_loc[0],y), (current_loc[0],y-1), False)]
                    step2 = [((current_loc[0],y-1), (current_loc[0],y), True)]
                    steps.extend(step1+step2)
                    first_movement=False
                else:
                    step1 = [((current_loc[0],y+1), (current_loc[0],y), False)]
                    step2 = [((current_loc[0],y), (current_loc[0],y-1), False)]
                    step3 = [((current_loc[0],y-1), (current_loc[0],y), True)]
                    steps.extend(step1+step2+step3)
            # last move to the escort, but make sure we have done earlier steps!
            if visited:
                steps.extend([((current_loc[0],y_target+1), (current_loc[0],y_target), False)])

        return steps
    
    
    #########################################################################################################################
    ############### THREE STEP MOVEMENTS
    #########################################################################################################################
    
    def three_step_horizontal(self,current_loc):
        # assuming the robot is with the escort
        steps=["check"]
        if current_loc[1] == 7: # we are at the same column as the I/O point is, no steps are needed.
            return steps
        else:
            direction = 1 if current_loc[1]>7 else -1 # defines on which side we are at (column-wise)
        
        steps.extend(self.columns_steps(current_loc,current_loc[1]-direction)) # one step left/right
        location_2 = (current_loc[0],current_loc[1]-direction)
        
        steps.extend(self.rows_steps(location_2,current_loc[0]-1)) # one step down
        location_3 = (current_loc[0]-1,current_loc[1]-direction)
        
        steps.extend(self.columns_steps(location_3,current_loc[1])) # one step to the item
        final_loc = steps[-1][1] # the location of the robot at the end

        return final_loc,steps
    
    def three_step_vertical(self,current_loc):
        # assuming the robot is with the escort
        steps=["check"]
        if current_loc[1] == 7: # we are at the same column as the I/O point is, no steps are needed.
            return steps
        else:
            direction = 1 if current_loc[1]>7 else -1 # defines on which side we are at (column-wise)
        
        # locations during the steps:
        location_2 = (current_loc[0]-1,current_loc[1])
        location_3 = (current_loc[0]-1,current_loc[1]-direction)
        
        # apply steps:
        steps.extend(self.rows_steps(current_loc,current_loc[0]-1)) # one step down
        steps.extend(self.columns_steps(location_2,current_loc[1]-direction)) # one step left/right
        steps.extend(self.rows_steps(location_3,current_loc[0])) # one step up

        final_loc = steps[-1][1] # the location of the robot at the end
        
        if final_loc[0] == 0: # we need to apply another final three step
            steps.extend(self.rows_steps(final_loc,1)) # one step towards the item
            steps.extend(self.columns_steps((final_loc[0]+1,final_loc[1]),final_loc[1]-direction)) # one step left/right
            steps.extend(self.rows_steps((final_loc[0]+1,final_loc[1]-direction),0)) # one step down
            final_loc = (0,final_loc[1]-direction)

        return final_loc,steps
    
    #########################################################################################################################
    ############### THREE STEP JOURNEY PLANNER
    #########################################################################################################################
    
    def three_step(self,robot_id):
        # by a given current_loc, calculate the needed three steps until we reach either column 7 or row 1
        last_valid_loc = self.robot_positions[robot_id-1][0]
        print(last_valid_loc)
        print("Robot id: %s, Item to take: %s, Currently taking %s"%(self.warehouse.iloc[last_valid_loc].robot.id,self.warehouse.iloc[last_valid_loc].robot.item_to_take,self.warehouse.iloc[last_valid_loc].robot.currently_taking))
        steps = []
        item_above, item_below, item_right, item_left = False,False,False,False
        
        # robot must be closer to the I/O
        items = [self.warehouse.iloc[(last_valid_loc)].item.number]
        if last_valid_loc[0] < 8: # item above
            item_above = True
            items.append(self.warehouse.iloc[(last_valid_loc[0]+1,last_valid_loc[1])].item.number)
        if last_valid_loc[0] > 0: # item below
            item_below = True
            items.append(self.warehouse.iloc[(last_valid_loc[0]-1,last_valid_loc[1])].item.number)
        if last_valid_loc[1] < 14: # item right
            item_right = True
            items.append(self.warehouse.iloc[(last_valid_loc[0],last_valid_loc[1]+1)].item.number)
        if last_valid_loc[1] > 0: # item left
            item_left = True
            items.append(self.warehouse.iloc[(last_valid_loc[0],last_valid_loc[1]-1)].item.number)
        
        print(items)
        
        if self.warehouse.iloc[last_valid_loc].robot.item_to_take not in items and self.warehouse.iloc[last_valid_loc].robot.currently_taking not in items:
            print("Item is not at the right place for robot %s; redefining the robot's route to item %s."%(robot_id,self.warehouse.iloc[last_valid_loc].robot.item_to_take))
            #self.manhattan_journey_to_item(robot_id,self.warehouse.iloc[last_valid_loc].robot.item_to_take)
            self.reroute(robot_id)
            return False
        else:
            # starting three steps - horizontal then vertical.
            print("Robot %s entered the three step! last valid location: %s"%(robot_id,str(last_valid_loc)))
            first_time = True
            
            escort_loc = self.return_to_escort(last_valid_loc,robot_id)
            print("escort_loc: %s"%str(escort_loc))
            if escort_loc: # we are checking that the escort is with the item
                steps.extend([(last_valid_loc,escort_loc,False)]) # one step to the escort
                last_valid_loc = escort_loc
            
            while last_valid_loc[1]!=7 and last_valid_loc[0]>0: # until we reach column 7 where the I/O point is, or until row is 1
                # one step towards the item
                if first_time and item_above: # make sure we can go to row 8
                    steps.append("check")
                    steps.extend(self.rows_steps(last_valid_loc,last_valid_loc[0]+1))
                    last_valid_loc = steps[-1][1] # update last valid location
                    first_time = False
                
                # apply three steps
                new_loc, current_steps = self.three_step_horizontal(last_valid_loc)
                last_valid_loc = new_loc # update last valid location
                steps.extend(current_steps)
                if last_valid_loc[0]>0: # maybe we are finished here, and the vertical step is unnecessary?
                    new_loc, current_steps = self.three_step_vertical(last_valid_loc)
                    last_valid_loc = new_loc # update last valid location
                    steps.extend(current_steps)
                    
        if steps:
            self.define_robot_path(robot_id,steps)
            print('Robot %s three-step path is: %s \n'%(robot_id,self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.path))
        else:
            item_to_take = self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.item_to_take
            print('No three step is needed for robot %s'%(robot_id))
            
            self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.currently_taking = item_to_take
            self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.item_to_take = None
            print("Robot %s is currently taking %s"%(robot_id,self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.currently_taking))
            self.five_step(robot_id) # do the necessary vertical five step
            return False
        return True

    #########################################################################################################################
    ############### FIVE STEP MOVEMENTS
    #########################################################################################################################
    
    def five_step_horizontal(self,current_loc):
        # assuming that the item is closer to the I/O in comparison to the robot
        last_valid_loc = current_loc
        steps = ["check"]
        if current_loc[1] - 7 > 0: #we are at the right side of the I/O
            steps.extend(self.rows_steps(current_loc,current_loc[0]+1)) #step up, first step
            steps.extend(self.columns_steps((current_loc[0]+1,current_loc[1]),current_loc[1]-2)) #two steps left
            steps.extend(self.rows_steps((current_loc[0]+1,current_loc[1]-2),current_loc[0])) #step down
            steps.extend(self.columns_steps((current_loc[0],current_loc[1]-2),current_loc[1]-1))#step to the item
        elif current_loc[1] - 7 < 0: #we are on the left side of the I/O
            steps.extend(self.rows_steps(current_loc,current_loc[0]+1)) #step up, first step
            steps.extend(self.columns_steps((current_loc[0]+1,current_loc[1]),current_loc[1]+2)) #two steps right
            steps.extend(self.rows_steps((current_loc[0]+1,current_loc[1]+2),current_loc[0])) #step down
            steps.extend(self.columns_steps((current_loc[0],current_loc[1]+2),current_loc[1]+1))#step to the item
        
        if steps: # any steps were created?
            final_loc = steps[-1][1] # the location of the robot at the end
        else: # no steps were created
            final_loc = current_loc
            
        return final_loc,steps
    
    
    def five_step_vertical(self,current_loc,side):
        # assuming that the item is closer to the I/O in comparison to the robot
        last_valid_loc = current_loc
        steps = ["check"]
        
        if current_loc[1]!=7:
            raise LocationError("The robot is not located on the I/O column!")
        
        if current_loc[0] > 1:#we are above the I/O on column 7
            if side == 1: # we are on the left side of the warehouse
                steps.extend(self.columns_steps(current_loc,current_loc[1]-1)) #step aside,, first step
                steps.extend(self.rows_steps((current_loc[0],current_loc[1]-1),current_loc[0]-2))#two steps down
                steps.extend(self.columns_steps((current_loc[0]-2,current_loc[1]-1),current_loc[1]))#back to the right wall
                steps.extend(self.rows_steps((current_loc[0]-2,current_loc[1]),current_loc[0]-1))#step to the item
            
            elif side == 2 or side == 3: # we are on the right side of the warehouse
                steps.extend(self.columns_steps(current_loc,current_loc[1]+1))#step aside, first step
                steps.extend(self.rows_steps((current_loc[0],current_loc[1]+1),current_loc[0]-2))#two steps down
                steps.extend(self.columns_steps((current_loc[0]-2,current_loc[1]+1),current_loc[1]))#back to left wall
                steps.extend(self.rows_steps((current_loc[0]-2,current_loc[1]),current_loc[0]-1))#step to the item
                
        if steps:
            final_loc = steps[-1][1] # the location of the robot at the end
        else: # no steps were created
            final_loc = current_loc
            
        return final_loc,steps
    
    #########################################################################################################################
    ############### FIVE STEP JOURNEY PLANNER
    #########################################################################################################################
    
    def five_step(self,robot_id):
        # By a given robot id, this function calculates the needed path towards the I/O point.
        last_valid_loc = self.robot_positions[robot_id-1][0]
        
        steps = []
        
        currently_taking = self.warehouse.iloc[last_valid_loc].robot.currently_taking
        side = self.warehouse.iloc[self.find_item_location(currently_taking)].item.side
        
        if last_valid_loc == (0,7): # are we in the IO?
            loc = self.around_IO()
            if loc: # Is there an item to exit around the IO point?
                print("The robot %s is about to exit an item unexpectedly!"%robot_id)
                if loc == (0,6): # one step left
                    steps.extend(self.columns_steps((0,7),6))
                elif loc == (0,8): # one step right
                    steps.extend(self.columns_steps((0,7),8))
                elif loc == (1,7): # one step down
                    steps.extend(self.rows_steps((1,7),0))
                self.warehouse.iloc[last_valid_loc].robot.path = steps

                # reset other robot plans
                item = self.warehouse.iloc[loc].item.number
                self.warehouse.iloc[last_valid_loc].robot.currently_taking = item
                if self.items_to_exit[item]: # another robot takes the item
                    other_robot = self.items_to_exit[item] # retrieve the robot_id who takes the item
                    other_robot_loc = self.robot_positions[other_robot-1][0]
                    self.items_to_exit[item] = robot_id
                    self.new_route(other_robot) # redefine the route of the other robot
                else: # no robot takes the item
                    self.items_to_exit[item] = robot_id
                
                    
                return False
                
                            
        
        
        if last_valid_loc[1] == 7: # case 1: the robot is in column 7 (the same column as the I/O)
            # Here we do vertical five steps until we reach the I/O point.
            item_above = self.warehouse.iloc[(last_valid_loc[0]+1,last_valid_loc[1])].item.number 
            item_below = self.warehouse.iloc[(last_valid_loc[0]-1,last_valid_loc[1])].item.number
            if item_above != currently_taking and item_below != currently_taking:
                print('Robot %s missed the item %s. Rerouting.'%(robot_id,currently_taking))
                self.reroute(robot_id)
                return False
            
            
            # At first, apply only one step towards the item, if the item is above the robot
            if item_above == currently_taking:
                steps.append("check")
                side = self.warehouse.iloc[(last_valid_loc[0]+1,last_valid_loc[1])].item.side
                steps.extend(self.rows_steps(last_valid_loc,last_valid_loc[0]+1))
                last_valid_loc = steps[-1][1] # update last valid location
            else: # item below
                side = self.warehouse.iloc[(last_valid_loc[0]-1,last_valid_loc[1])].item.side
            
            # all is set; now start the needed vertical five steps.
        
            while last_valid_loc[0]>1: # until the robot is only 1 row above the I/O
                last_valid_loc, current_steps = self.five_step_vertical(last_valid_loc, side) # movements according to the item's side!
                steps.extend(current_steps)
                
        
        elif last_valid_loc[0]<=1: # case 2: the robot is in row 1 or 0 (one row above the IO)
            item_left = self.warehouse.iloc[(last_valid_loc[0],last_valid_loc[1]-1)].item.number
            item_right = self.warehouse.iloc[(last_valid_loc[0],last_valid_loc[1]+1)].item.number
            currently_taking = self.warehouse.iloc[last_valid_loc].robot.currently_taking

            if last_valid_loc[1]>7 and item_right == currently_taking: # one step right is needed towards the item
                steps.extend(self.columns_steps(last_valid_loc,last_valid_loc[1]+1))
                last_valid_loc = steps[-1][1] # update last valid location

            elif last_valid_loc[1]<7 and item_left == currently_taking: # one step left is needed towards the item
                steps.extend(self.columns_steps(last_valid_loc,last_valid_loc[1]-1))
                last_valid_loc = steps[-1][1] # update last valid location



            elif item_left != currently_taking and item_right != currently_taking: # there is a problem
                print('Robot %s missed the item %s. Rerouting.'%(robot_id,currently_taking))
                self.reroute(robot_id)
                return False
                    
            # all is set; start the horizontal five steps.
            print(last_valid_loc[1])
            while last_valid_loc[1]<6 or last_valid_loc[1]>8: # until the robot is only 1 row above the I/O
                new_loc, current_steps = self.five_step_horizontal(last_valid_loc) # movements according to the item's side!
                last_valid_loc = new_loc
                steps.extend(current_steps)
            
            #now, at the end, we need one vertical five-step to (0,7) if the robot is two rows above the I/O
            if last_valid_loc == (2,7):
                side = self.warehouse.iloc[(2,7)].item.side
                steps.extend(self.rows_steps(last_valid_loc,last_valid_loc[0]+1)) # one step above
                last_valid_loc, current_steps = self.five_step_vertical(last_valid_loc, side) # movements according to the item's side!
                steps.extend(current_steps)

        #else: # there is a problem
        #    raise LocationError("The item is not located at the right place! Item's location: %s, robot's location: %s"%(item_above,self.warehouse.iloc[last_valid_loc].robot.item_to_take))
        
        # define the planned steps
        self.define_robot_path(robot_id,steps)
        print('Robot %s five-step path is: %s \n'%(robot_id,self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.path))
        
    
    #########################################################################################################################
    ############### TO THE NEXT ITEM JOURNEY PLANNER
    #########################################################################################################################
    
    def to_next_item(self,robot_id,item_number):
        # Use this function only if the robot has turned over the item to the I/O point & is in the location: (1,7) (0,6) or (0,8)
        item_location = self.find_item_location(item_number)
        robot_loc = self.robot_positions[robot_id-1][0] # retrieve current robot's position
        print("Current robot location: %s"%(str(robot_loc)))
        side = 1 if item_location[1] <= 7 else 2
        
        #if robot_loc != (1,7) and robot_loc != (0,6) and robot_loc != (0,8):
        #    self.manhattan_journey_to_item(robot_id,item_number)
        #    return False
        steps = []
        if self.warehouse.iloc[robot_loc].item.number != 0:
            escort_loc = self.return_to_escort(robot_loc,robot_id)
            steps.extend([(robot_loc,escort_loc,False)])
            old_robot_loc = robot_loc
            robot_loc = escort_loc
        else:
            old_robot_loc = robot_loc
            
        print(steps)
        
        ###### NEED TO IMPLEMENT BACK TO ESCORT FUNCTION

        if robot_loc[0]>0: # Going back from the escort
            steps.extend(self.rows_steps(robot_loc,0)) #robot goes to row 0
        #elif robot_loc[1]>7 or robot_loc[1]<7: 
        #    steps.extend(self.columns_steps(robot_loc_2,7))
        
        row_below_item = 0 if item_location[0] == 0 else item_location[0]-1
        
        p = 1 if self.robot_side[robot_id]==1 else -1
        
        if item_location[1] == 0 or item_location[1] == 14:
            item_location = (item_location[0],item_location[1]+p)
        
        if side == 1: # next item is at the left side of the warehouse
            steps.extend(self.columns_steps((0,robot_loc[1]),0))              # go to the left wall of the warehouse: (0,0)
            steps.extend(self.rows_steps((0,0),row_below_item))                    # go to one row below the item (if possible)
            steps.extend(self.columns_steps((row_below_item,0),item_location[1]))  # go to the item's column
            
        else: # next item is at the right side of the warehouse
            steps.extend(self.columns_steps((0,robot_loc[1]),14))             # go to the right wall of the warehouse: (0,14)
            steps.extend(self.rows_steps((0,14),row_below_item))                   # go to one row below the item (if possible)
            steps.extend(self.columns_steps((row_below_item,14),item_location[1])) # go to the item's column
        
        #elif side == 3: # next item is in the I/O column
        #    # to avoid collisions, let the robot reach the points: (0,0) --> (8,0) --> (8,7) --> (item_row-1,7)
        #    steps.extend(self.columns_steps((0,robot_loc[1]),0))              # go to the left wall of the warehouse: (0,0)
        #    steps.extend(self.rows_steps((0,0),8))                                 # go to the edge point (8,0)
        #    steps.extend(self.columns_steps((0,8),7))                              # go to the I/O column
        #    steps.extend(self.rows_steps((8,7),row_below_item))                    # go to one row below the item
        
        
        self.warehouse.iloc[old_robot_loc].robot.item_to_take = item_number
        self.warehouse.iloc[old_robot_loc].robot.currently_taking = None
        print("Robot %s is now taking the item %s"%(robot_id,item_number))
        self.items_to_exit[item_number] = robot_id
        self.define_robot_path(robot_id,steps)
    
    #########################################################################################################################
    ############### APPLYING ROBOT STEPS
    #########################################################################################################################
    
    def new_route(self, robot_id):
        # in case we want to set a new route for a specific robot
        if self.items_to_exit: # there are still items to exit
            if self.robot_side[robot_id] == 1:
                if len(self.distances_left) > 0:
                    next_item = random.choice(self.distances_left)[0]
                else:
                    return False # no items

            else:
                if len(self.distances_right) > 0:
                    next_item = random.choice(self.distances_right)[0]
                else:
                    return False # no items

            robot_loc = self.robot_positions[robot_id-1][0]

            if self.warehouse.iloc[robot_loc].robot.item_to_take:
                self.items_to_exit[self.warehouse.iloc[robot_loc].robot.item_to_take] = False
            elif self.warehouse.iloc[robot_loc].robot.currently_taking:
                self.items_to_exit[self.warehouse.iloc[robot_loc].robot.currently_taking] = False
            
            self.manhattan_journey_to_item(robot_id,next_item)
            # if the robot is exactly in the place to start 3 or 5 steps:
            if self.warehouse.iloc[robot_loc].robot.path[0] == 'check':
                self.warehouse.iloc[robot_loc].robot.item_to_take = None
                self.warehouse.iloc[robot_loc].robot.currently_taking = next_item
            else: # the robot will attend the item
                self.warehouse.iloc[robot_loc].robot.item_to_take = next_item
                self.warehouse.iloc[robot_loc].robot.currently_taking = None
            self.items_to_exit[next_item] = robot_id  

            print("We are replacing the item for robot %s to item %s"%(robot_id,next_item))
            return True
        
        return False # no items
    
    def escort_in_target(self, location, robot_id):
        if self.warehouse.iloc[location].item.number == 0 and self.warehouse.iloc[location].item.robot_id != robot_id: # Escort is indeed in target
            new_route = self.new_route(robot_id)
            if new_route:
                return new_route
            else:
                return True
        else:
            return False

    def apply_robot_step(self, robot_id, fictitious=False):
        # apply the given step to a robot_id 
        robot_loc = self.robot_positions[robot_id-1][0]

        if fictitious: # if the applied step is fictitious
            self.robots_moves[robot_id].append((robot_loc, robot_loc, False)) # add a fictitious move
            print('Fictitious move for robot %s applied'%(robot_id))
            print('Robot %s next 3 locations: %s'%(robot_id,str(self.warehouse.iloc[robot_loc].robot.path[0:3])))
        else: # we can start!
            step = self.warehouse.iloc[robot_loc].robot.path[0]
            if step == "check":
                if not self.location_check(robot_id):
                    return False # stop here if the robot isn't located in the right place for 3/5 steps
                else:
                    del self.warehouse.iloc[robot_loc].robot.path[0]
                    
            current_loc, to_loc, with_item = self.warehouse.iloc[robot_loc].robot.path[0]
            print('Robot %s to_loc: %s, next 3 locations: %s'%(robot_id,str(to_loc),str(self.warehouse.iloc[robot_loc].robot.path[1:4])))
        
            current_cell, to_cell = self.warehouse.iloc[current_loc], self.warehouse.iloc[to_loc]
            
            
            if current_loc == to_loc: # in case origin location equals to destination location
                self.robots_moves[robot_id].append((current_loc, current_loc, False)) # apply fictitious move
                
            elif self.warehouse.iloc[to_loc].robot != '':
                print('Robot %s stopped at %s in order to avoid crash.'%(robot_id,str(to_loc)))
                #self.robots_moves[robot_id].append((current_loc, current_loc, False)) # add a fictitious move
                self.escape(robot_id,to_loc)
                with_item,new_loc,next_new_loc = self.warehouse.iloc[current_loc].robot.path[0]
                self.robots_moves[robot_id].append((new_loc, next_new_loc, False))
                return False
                
            elif self.escort_in_target(to_loc, robot_id): # escort in target?
                # freeze for 3 steps (this one and the next 2)
                print('Escort in target. Freezing robot %s in location %s for 3 time units'%(robot_id,str(current_loc)))
                for i in range(2):
                    self.define_robot_path(robot_id,(current_loc, current_loc, False),overwrite=False)
                self.robots_moves[robot_id].append((current_loc, current_loc, False)) # add a fictitious move
                return False
                
            else:
                self.warehouse.iloc[to_loc].assignRobot(current_cell.robot) # assign the robot to the destination cell
                self.warehouse.iloc[current_loc].assignRobot('') # empty the current cell from the robot
                self.robots_moves[robot_id].append((with_item, current_loc, to_loc))
                if with_item: # robot is moving to the new location with the item
                    if self.warehouse.iloc[to_loc].item.number != 0: # if the destination isn't an escort
                        raise ItemCrashError("Robot %s: Instead of an escort, In the target location %s there is already an item"%(robot_id,str(to_loc)))
                    else:
                        escort = self.warehouse.iloc[to_loc].item
                        self.warehouse.iloc[to_loc].assignItem(current_cell.item) # assign the item to the destination cell
                        self.warehouse.iloc[current_loc].assignItem(escort) # now the current cell has an escort
                
            del self.warehouse.iloc[to_loc].robot.path[0] # delete the planned step for the robot that it is
                                                          # not at to_loc
            
            
    
    #########################################################################################################################
    ############### MAIN PROGRAM FUNCTIONS
    #########################################################################################################################
        
    def running_first_time(self):
        
        for robot_id in range(1,6):
            i = 1 if robot_id%2==0 else -1
            if self.robot_side[robot_id] == 1: # item according to the robot side
                item = self.distances_left[i*robot_id-1][0]
            else:
                item = self.distances_right[i*robot_id-1][0]
            
            self.manhattan_journey_to_item(robot_id,item) # robot_id = i+1
            self.items_to_exit[item] = robot_id
            print("Robot %s path is: %s"%(robot_id,str(self.warehouse.iloc[self.robot_positions[robot_id-1][0]].robot.path)))    
                    
    
    def can_proceed(self, robot_id):
        # identify the robots in the area of 4X4 around the I/O, that are currently taking an item 
        # and find the robot with the minimum remaining steps towards the I/O that can proceed with it.
        area = [(i,j) for i in range(6) for j in range(4,11)]
        
        robot_locs = [location for location in area if self.warehouse.iloc[location].robot != '']
        candidates = []
        
        for location in robot_locs: # find pairs of (robot_id,distance)
            if self.warehouse.iloc[location].robot.currently_taking:
                candidate_id = self.warehouse.iloc[location].robot.id
                distance_left = location[0] + abs(location[1]-7) # rows+(columns-7) is the distance from the IO
                candidates.append((candidate_id,distance_left))
        
        if not candidates:
            return True  # no robots around the I/O, proceed with current robot
        else:
            candidate = sorted(candidates, key=lambda x: x[1])[0][0]  # return the robot_id with minimum distance
            if candidate == robot_id:  # the candidate is this robot_id?
                return True
            else:
                return False
            
    ## Collisions handling
    
    def escape(self, robot_id, other_robot_next_loc):
        robot_loc = self.robot_positions[robot_id-1][0]
        if robot_id == self.warehouse.iloc[other_robot_next_loc].robot.id:
            print("Same robot - aborting escape.")
            return False
        
        # define direction
        if robot_loc[0] == 0 or robot_loc[1] == 0:
            direction = 1
        elif robot_loc[0] == 8 or robot_loc[1] == 14:
            direction = -1
        else:
            direction = random.choice([1,-1])

        steps = []
        
        if robot_loc[1] - other_robot_next_loc[1] == 0: # on the same column, so move to another column
            print("Robot %s entered - same column"%robot_id)
            temporary_location = (robot_loc[0],robot_loc[1]+direction)
            steps.extend(self.columns_steps(robot_loc,robot_loc[1]+direction)) # move one column
            steps.extend([(temporary_location, temporary_location, False) for i in range(3)]) # freeze for 3 time units
            steps.extend(self.columns_steps(temporary_location,robot_loc[1])) # return to column
        elif robot_loc[0] - other_robot_next_loc[0] == 0: # on the same row, so move by rows
            print("Robot %s entered - same row"%robot_id)
            temporary_location = (robot_loc[0]+direction,robot_loc[1])
            steps.extend(self.rows_steps(robot_loc,robot_loc[0]+direction))
            steps.extend([(temporary_location, temporary_location, False) for i in range(3)]) # freeze for 3 time units
            steps.extend(self.rows_steps(temporary_location, robot_loc[0]))
        
        # otherwise - freeze.
        if steps:
            print("Robot %s escaped with the following steps:\n%s"%(robot_id,str(steps)))
        
        for step in reversed(steps): # define robot path inserts the steps like a stack
            self.define_robot_path(robot_id,step,overwrite=False)
        
        
        
        return True
    
    def reroute(self,robot_id):
        # when needed, this function will reset the robot's planned steps.
        #self.calculate_positions()
        robot_loc = self.robot_positions[robot_id-1][0]
        currently_taking = self.warehouse.iloc[robot_loc].robot.currently_taking
        item_to_take = self.warehouse.iloc[robot_loc].robot.item_to_take
        
        if currently_taking:
            self.manhattan_journey_to_item(robot_id,currently_taking)
            self.warehouse.iloc[robot_loc].robot.currently_taking = None
            self.warehouse.iloc[robot_loc].robot.item_to_take = currently_taking
        else:
            self.manhattan_journey_to_item(robot_id,item_to_take)
        
        

        
        print("Reroute completed for robot %s toward the item %s"%(robot_id,currently_taking))
    
    def location_check(self,robot_id):
        # check if the robot is near the item for 3 step or 5 step
        location = self.robot_positions[robot_id-1][0]
        above = 8 if location[0] == 8 else location[0]+1
        below = 0 if location[0] == 0 else location[0]-1
        left = 0 if location[1] == 0 else location[1]-1
        right = 14 if location[1] == 14 else location[1]+1
        
        item_location = self.find_item_location(self.warehouse.iloc[location].robot.currently_taking)
        
        if item_location[0] in [above,below] or item_location[1] in [left,right]:
            return True
        else:
            print("Robot %s should start a 3-step or a 5-step, and its location is incorrect. Rerouting..."%robot_id)
            self.reroute(robot_id)
            return False
        
    def return_to_escort(self, location, robot_id):
        above = (8,location[1]) if location[0] == 8 else (location[0]+1,location[1])
        below = (0,location[1]) if location[0] == 0 else (location[0]-1,location[1])
        left = (location[0],0) if location[1] == 0 else (location[0],location[1]-1)
        right = (location[0],14) if location[1] == 14 else (location[0],location[1]+1)
        
        for loc in [above,below,left,right]:
            if self.warehouse.iloc[loc].item.number == 0:
                if self.warehouse.iloc[loc].item.robot_id == robot_id:
                    return loc
        
        return False # the escort is with the robot
        
    def around_robot(self, robot_id):
        location = self.robot_positions[robot_id-1][0]
        above = (8,location[1]) if location[0] == 8 else (location[0]+1,location[1])
        below = (0,location[1]) if location[0] == 0 else (location[0]-1,location[1])
        left = (location[0],0) if location[1] == 0 else (location[0],location[1]-1)
        right = (location[0],14) if location[1] == 14 else (location[0],location[1]+1)
        
        robots_around = []
        
        for loc in [above,below,left,right]:
            if self.warehouse.iloc[loc].robot != '': # robot around the IO?
                return loc # return the location of the robot
        
        return False
            
    def around_IO(self):
        # this function checks if there is an item to exit around the IO
        
        for loc in [(0,6),(0,8),(1,7)]:
            if loc in self.items_to_exit_positions:
                print("Item around IO: %s"%str(loc))
                return loc
        
        return False
        
    
    def final(self,robot_id):
        
        print("Robot %s finished his tasks!!!!!!!!!!!!!!!!!!"%robot_id)
        robot_loc = self.robot_positions[robot_id-1][0]
        self.warehouse.iloc[robot_loc].robot.currently_taking = None
        self.warehouse.iloc[robot_loc].robot.item_to_take = None
        if self.warehouse.iloc[robot_loc].item.number != 0:
            escort_loc = self.return_to_escort(robot_loc,robot_id)
            step = (robot_loc,escort_loc,False)
            old_robot_loc = robot_loc
            robot_loc = escort_loc
        else:
            old_robot_loc = robot_loc
        
        ###### NEED TO IMPLEMENT BACK TO ESCORT FUNCTION

        
        if self.robot_side[robot_id] == 1:
            loc = self.robot_final_positions_left[0]
            item = self.warehouse.iloc[loc].item.number
            self.manhattan_journey_to_item(robot_id,item,final=True,final_loc=robot_loc)
            #if loc == (4,0):
            #    steps.extend(self.rows_steps(robot_loc,3)) # to row 3
            #    steps.extend(self.columns_steps((3,robot_loc[1]),0)) # to (4,0)
            #elif loc == (0,0):
            #    steps.extend(self.columns_steps(robot_loc,0)) # to column 0
            #else:
            #    steps.extend(self.columns_steps(robot_loc,4)) # to column 4
            del self.robot_final_positions_left[0]
        else:
            loc = self.robot_final_positions_right[0]
            item = self.warehouse.iloc[loc].item.number
            self.manhattan_journey_to_item(robot_id,item,final=True,final_loc=robot_loc)
            #if loc[0] == 8:
            #    steps.extend(self.columns_steps(robot_loc,11)) # to column 11
            #    steps.extend(self.rows_steps((robot_loc[0],11),4)) # to row 4
            #    steps.extend(self.columns_steps((4,11),14)) # to (4,14)
            #else:
            #    steps.extend(self.columns_steps(robot_loc,14)) # to column 14
            del self.robot_final_positions_right[0]
        
        #if robot_loc_2[0]>0: # Going back from the escort
        #    steps.extend(self.rows_steps(robot_loc_2,0)) #robot goes to row 0
        #elif robot_loc_2[1]>7 or robot_loc_2[1]<7: 
        #    steps.extend(self.columns_steps(robot_loc_2,7))
        
        #self.define_robot_path(robot_id,step,overwrite=False)
        
        print("Robot %s is going to %s\nFull final path:"%(robot_id,str(loc)))
        print(self.warehouse.iloc[old_robot_loc].robot.path)
        
        return True
        

## 1.5. Main Function - change

<div dir=rtl>
סדר הפונקציה הוא כזה: מתחילים קודם כל בזמן 1. בזמן הזה מחלקים לרובוטים את הפריטים להוצאה. לאחר מכן בודקים האם כל רובוט יכול לבצע את הצעד המתכונן שלו.<br>
    אם כן - הוא מבצע אותו. אחרת - הוא קופא במקום ומוסיפים צעד פיקטיבי.<br>
    אם אין פריטים להוצאה, אז זה אומר שהגענו לאחד מהמצבים הבאים: או שהרובוט סיים להגיע לפריט ועכשיו צריך להתחיל צעדי 3, או שהרגע הסתיימו צעדי ה3 וצריך להתחיל צעדי 5, או שסיימנו צעדי 5 וצריך להתחיל ללכת לכיוון הפריט הבא בתור.<br>
        כרגע אין התמודדות מיוחדת עם התנגשויות - כדאי להריץ בpycharm כדי לראות איך כל התוכנית מתקדמת צעד אחרי צעד
    </div>

In [972]:
def highlight_cells(x):
    df = x.copy()
    df.loc[:,:] = '' 
    #set particular cell colors for robots
    for robot_pos in j.robot_positions:
        df.iloc[robot_pos[0]] = 'background-color: yellow;'
    
    for item_pos in j.items_to_exit_positions:
        if df.iloc[robot_pos[0]] == 'background-color: yellow;':
            df.iloc[item_pos] += ' font-weight: bold'
        else:
            df.iloc[item_pos] = 'font-weight: bold'
    
    df.iloc[0,7] = 'background-color: blue; color: white;'

    return df  

In [978]:
j = Warehouse('wh8.p', 'items_list.p')

[((3, 2), 1), ((4, 8), 2), ((7, 0), 3), ((8, 8), 4), ((8, 9), 5)]


In [979]:
j.warehouse.style.apply(highlight_cells, axis=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[213,]","[215,]","[114,]","[186 to exit,]","[119,]","[171,]","[138,]","[110,]","[136,]","[196 to exit,]","[203,]","[147,]","[170,]","[194 to exit,]","[163,]"
1,"[177,]","[197,]","[139 to exit,]","[158,]","[160,]","[220 to exit,]","[121,]","[185,]","[142,]","[199,]","[181,]","[229 to exit,]","[184,]","[190,]","[228,]"
2,"[218,]","[135,]","[212,]","[140,]","[117,]","[157,]","[130,]","[107,]","[132,]","[172,]","[102,]","[169,]","[146,]","[223,]","[152,]"
3,"[178,]","[217,]","[0,Robot 1]","[174,]","[188 to exit,]","[204,]","[156,]","[175,]","[201,]","[128,]","[179,]","[173,]","[112,]","[209,]","[118 to exit,]"
4,"[208,]","[116,]","[115,]","[164,]","[127,]","[205 to exit,]","[219,]","[221 to exit,]","[0,Robot 2]","[153,]","[137,]","[192 to exit,]","[141,]","[165 to exit,]","[111,]"
5,"[161 to exit,]","[125,]","[124,]","[200 to exit,]","[129,]","[183 to exit,]","[148 to exit,]","[198,]","[225,]","[206,]","[144,]","[143,]","[150,]","[103,]","[101,]"
6,"[149,]","[166,]","[226,]","[133,]","[120,]","[109,]","[106,]","[154,]","[224,]","[162,]","[176,]","[122 to exit,]","[195 to exit,]","[222,]","[168,]"
7,"[0,Robot 3]","[151,]","[193,]","[182,]","[155,]","[210,]","[159,]","[131,]","[104,]","[134,]","[123,]","[180 to exit,]","[126,]","[167,]","[191,]"
8,"[214,]","[227 to exit,]","[207,]","[187 to exit,]","[100,]","[105,]","[211,]","[189 to exit,]","[0,Robot 4]","[0,Robot 5]","[145,]","[108,]","[113 to exit,]","[216 to exit,]","[202 to exit,]"


## Main Program

In [967]:
def main_program(wh):
    time = 1
    beginning = True
    #dataframe_image.export(wh.warehouse.style.apply(highlight_cells, axis=None),'running/time_%s.png'%time)
    while wh.items_to_exit: # as long as we have items to exit
        
        #### RUNNING FIRST TIME ####
        if beginning: # time to initiate this project!
            wh.running_first_time()
            beginning=False
            
        else:         # let's get the warehouse to work!
            print('\nTime %s\n-----------'%(time))
            for i in range(5):
                print("Robot %s is at %s, intended to take the item %s, currently taking %s"%(i+1,wh.robot_positions[i][0],
                                                                                              wh.warehouse.iloc[wh.robot_positions[i][0]].robot.item_to_take,
                                                                                              wh.warehouse.iloc[wh.robot_positions[i][0]].robot.currently_taking))
            print("\n")
            print("Robot locations: %s"%str(wh.robot_positions))
            
            steps_to_apply = {1: False, 2: False, 3: False, 4: False, 5: False}
            
            ##############################################################
            ################## Check if an item to exit is in the I/O
            ##############################################################
            if wh.warehouse.iloc[(0,7)].item.number in wh.items_to_exit: # An item to exit is in the I/O?
                robot_id = wh.items_to_exit[wh.warehouse.iloc[(0,7)].item.number]
                robot_loc = wh.robot_positions[robot_id-1][0]
                wh.exit_item(robot_id,time)
                print("Exited items so far: %s\nLeft items:"%(str(wh.exited_items)))
                print(wh.items_to_exit)
                print("Items in the left warehouse: %s\nItems in the right warehouse:%s"%(str(wh.distances_left),str(wh.distances_right)))
                if wh.items_to_exit and robot_id: # there are still items to exit
                    if wh.robot_side[robot_id] == 1:
                        if len(wh.distances_left) > 0:
                            next_item = wh.distances_left[-1][0]
                            print("Chosen item: %s out of %s"%(next_item,len(wh.items_to_exit)))
                            wh.to_next_item(robot_id,next_item)
                            wh.warehouse.iloc[robot_loc].robot.item_to_take = next_item
                            wh.warehouse.iloc[robot_loc].robot.currently_taking = None
                            print("Robot %s is now taking the item %s"%(robot_id,next_item))
                            wh.items_to_exit[next_item] = robot_id
                            steps_to_apply[robot_id] = True
                        else:
                            steps_to_apply[robot_id] = wh.final(robot_id)

                    else:
                        if len(wh.distances_right) > 0:
                            next_item = wh.distances_right[-1][0]
                            print("Chosen item: %s out of %s"%(next_item,len(wh.items_to_exit)))
                            wh.to_next_item(robot_id,next_item)
                            wh.warehouse.iloc[robot_loc].robot.item_to_take = next_item
                            wh.warehouse.iloc[robot_loc].robot.currently_taking = None
                            print("Robot %s is now taking the item %s"%(robot_id,next_item))
                            wh.items_to_exit[next_item] = robot_id
                            steps_to_apply[robot_id] = True
                        else:
                            steps_to_apply[robot_id] = wh.final(robot_id)

            ##############################################################
            ################## Going over each robot in this time unit
            ##############################################################
            for robot_id in range(1,6): # let's go over each robot
                
                robot_loc = wh.robot_positions[robot_id-1][0]
                
                if wh.warehouse.iloc[robot_loc].robot.path: # the robot has some steps to do?
                    if wh.warehouse.iloc[robot_loc].robot.path[0] == "check": # is it a 3-step or a 5-step?
                        if wh.location_check(robot_id): # it's ok
                            del wh.warehouse.iloc[robot_loc].robot.path[0]
                        else:
                            print('Location is not correct for robot %s'%robot_id)
                            continue # stop here if the robot isn't located in the right place for 3/5 steps
                            
                    step = wh.warehouse.iloc[robot_loc].robot.path[0]
                    next_loc = step[1]
                    
                    if robot_loc == next_loc:
                        # planned fictitious move
                        steps_to_apply[robot_id] = True
                        
                    elif wh.warehouse.iloc[next_loc].robot != '': # robot in target?
                        print("Robot %s stopped - robot in destination."%robot_id)
                        # rescue from collision
                        if wh.warehouse.iloc[next_loc].robot.path: # the robot in destination has steps to do?
                            other_robot_next_loc = wh.warehouse.iloc[next_loc].robot.path[0][1]
                            if next_loc == other_robot_next_loc: # the target of that robot is the same as this robot?
                                wh.escape(robot_id,other_robot_next_loc)
                                steps_to_apply[robot_id] = True
                                print("Now robot %s has the following 3 steps: %s"%(robot_id,str(wh.warehouse.iloc[robot_loc].robot.path[:3])))
                            else:
                                # just freeze for 3 time units
                                for i in range(3):
                                    wh.define_robot_path(robot_id,(robot_loc, robot_loc, False),overwrite=False)
                                print("Robot %s freezes for 3 time units"%robot_id)
                     
                    else: # maybe there is an escort in target?
                        if wh.escort_in_target(next_loc, robot_id): # is there an escort in target?
                            belongs_to = wh.warehouse.iloc[next_loc].item.robot_id
                        
                            if robot_id == belongs_to: # the escort belongs to the robot, safely proceed
                                steps_to_apply[robot_id] = True

                            else: # this is someone else's escort!
                                print("Escort issue with robot %s"%robot_id)
                                #if wh.warehouse.iloc[next_loc].robot == '': # the escort is without the robot?
                                #    print("There is a free escort in target. Freezing for 3 time units")
                                #    # just freeze for 3 time units
                                #    for i in range(3):
                                #        wh.define_robot_path(robot_id,(robot_loc, robot_loc, False),overwrite=False)
                                if not wh.warehouse.iloc[next_loc].robot == '':
                                    other_robot_next_loc = wh.warehouse.iloc[next_loc].robot.path[0][1]
                                    print("Robot %s Stopped, the escort in %s isn't his. adding fictitious move. Escaping."%(robot_id,str(next_loc)))
                                    wh.escape(robot_id,other_robot_next_loc)
                                    print("Now robot %s has the following 3 steps: %s"%(robot_id,str(wh.warehouse.iloc[robot_loc].robot.path[:3])))
                                
                                steps_to_apply[robot_id] = True
                                
                        else: # in case there is neither robot nor escort is in target
                            currently_taking = wh.warehouse.iloc[robot_loc].robot.currently_taking
                            if 0 <= next_loc[0] <= 4 and 5 <= next_loc[1] < 9 and currently_taking:
                                # entering the 'restricted zone'
                                if wh.can_proceed(robot_id):
                                    steps_to_apply[robot_id] = True # robot can proceed
                                else: # robot cannot proceed
                                    print('Robot %s Stopped because of load around the I/O point.'%robot_id)
                                    loc = wh.around_robot(robot_id)
                                    if loc: # if there is another robot, escape!!!
                                        print("Robot %s is next to another robot near the IO restricted zone. Escaping..."%robot_id)
                                        if wh.warehouse.iloc[loc].item.number == 0: # we make sure we do the escape when the robot is with the escort
                                            if wh.warehouse.iloc[robot_loc].robot.currently_taking: # rerouting the robot to the item
                                                item = wh.warehouse.iloc[robot_loc].robot.currently_taking
                                                wh.warehouse.iloc[robot_loc].robot.currently_taking = None
                                                wh.warehouse.iloc[robot_loc].robot.item_to_take = item
                                            else:
                                                item = self.warehouse.iloc[robot_loc].robot.item_to_take
                                            
                                            if wh.manhattan_journey_to_item(robot_id,item): # maybe manhattan journey is not needed?
                                                if wh.warehouse.iloc[robot_loc].robot.item_to_take: # the robot is in item to take mode?
                                                    wh.warehouse.iloc[robot_loc].robot.currently_taking = wh.warehouse.iloc[robot_loc].robot.item_to_take
                                                    wh.warehouse.iloc[robot_loc].robot.item_to_take = None
                                                    print("Now robot %s is currently taking %s"%(robot_id,wh.warehouse.iloc[robot_loc].robot.currently_taking))
                                            
                                            if type(wh.warehouse.iloc[robot_loc].robot.path[0][1]) != str:
                                                to_loc = wh.warehouse.iloc[robot_loc].robot.path[0][1]
                                                if wh.warehouse.iloc[to_loc].robot != '': # is there a robot in the new next planned location?
                                                    wh.escape(robot_id,loc)
                                        
                            else:
                                steps_to_apply[robot_id] = True
                                
                    
               
                else: # the robot doesn't have some steps to do?
                            
                    
                    if wh.warehouse.iloc[robot_loc].robot.item_to_take: # the robot finished the manhattan trip
                        # now we are at three step
                        if not wh.three_step(robot_id): # if we have failed to plan three-step movements
                            steps_to_apply[robot_id] = True
                            continue # The needed steps before three step are planned; continue to the next robot

                        item_number = wh.warehouse.iloc[robot_loc].robot.item_to_take
                        wh.warehouse.iloc[robot_loc].robot.robot_is_taking(item_number)
                        print("Robot %s is currently taking %s"%(robot_id,item_number))
                        steps_to_apply[robot_id] = True

                    elif wh.warehouse.iloc[robot_loc].robot.currently_taking: # check if the item's attended the IO
                        # now we are at five step
                        wh.five_step(robot_id)
                        # apply the steps now
                        steps_to_apply[robot_id] = True
                        

            for robot_id in steps_to_apply:
                if steps_to_apply[robot_id]: # apply the step here 
                    wh.apply_robot_step(robot_id)
                else: # apply fictitious step here
                    wh.apply_robot_step(robot_id,fictitious=True)
            
            
            ## Update positions
            wh.calculate_positions()
            wh.calculate_distance_from_IO()
                            


        
            
            
            ## To the next time unit
            time += 1
            
            print([len(wh.robots_moves[robot]) for robot in wh.robots_moves])
        
            
            #if time > 117:
            #    dataframe_image.export(wh.warehouse.style.apply(highlight_cells, axis=None),'running/time_%s.png'%time)
    
    print ("It took %s time units to exit all items."%(time))
    return wh

In [968]:
def run_and_export_to_pickle(wh):
    current = Warehouse(wh, 'items_list.p')
    example_number = wh[:3] # retrieves the "whX" name where X is the number of the warehouse set example
    current = main_program(current)
    
    ## Robot moves
    steps = []
    for i in range(1,6):
        steps.append(current.robots_moves[i])
        
    # write the pickle file
    with open('robots_moves_%s.p'%(example_number),'wb') as infile:
        p.dump(steps,infile) # automatically closes the connection with the file
    
    print("\n\n Robot moves were exported to pickle successfully")
    
    ## Extractions
    extractions = current.exited_items.items()
    extractions = sorted(extractions, key=lambda x: x[1])
    

    # write the pickle file
    with open('extractions_%s.p'%(example_number),'wb') as infile:
        p.dump(extractions,infile)
        
    print("\n\n Extractions were exported to pickle successfully")

In [969]:
run_and_export_to_pickle('wh9.p')

[((2, 8), 1), ((3, 1), 2), ((3, 3), 3), ((5, 4), 4), ((6, 10), 5)]
Manhattan journey - item 192 is in (4, 1), target location for robot 1: (4, 1)
escort_loc: False
Overwritten path for robot 1
[((2, 8), (3, 8), False), ((3, 8), (2, 8), True), ((2, 8), (3, 8), False), ((3, 8), (3, 7), False), ((3, 7), (3, 8), True), ((3, 8), (3, 7), False), ((3, 7), (3, 6), False), ((3, 6), (3, 7), True), ((3, 7), (3, 6), False), ((3, 6), (3, 5), False), ((3, 5), (3, 6), True), ((3, 6), (3, 5), False), ((3, 5), (3, 4), False), ((3, 4), (3, 5), True), ((3, 5), (3, 4), False), ((3, 4), (3, 3), False), ((3, 3), (3, 4), True), ((3, 4), (3, 3), False), ((3, 3), (3, 2), False), ((3, 2), (3, 3), True), ((3, 3), (3, 2), False), ((3, 2), (3, 1), False), ((3, 1), (3, 2), True), ((3, 2), (3, 1), False)]
Robot 1 will take 192 out
Robot 1 path is: [((2, 8), (3, 8), False), ((3, 8), (2, 8), True), ((2, 8), (3, 8), False), ((3, 8), (3, 7), False), ((3, 7), (3, 8), True), ((3, 8), (3, 7), False), ((3, 7), (3, 6), False

[2, 2, 2, 2, 2]

Time 3
-----------
Robot 1 is at (2, 8), intended to take the item 192, currently taking None
Robot 2 is at (3, 1), intended to take the item 195, currently taking None
Robot 3 is at (3, 3), intended to take the item 205, currently taking None
Robot 4 is at (5, 4), intended to take the item 148, currently taking None
Robot 5 is at (6, 10), intended to take the item 221, currently taking None


Robot locations: [((2, 8), 1), ((3, 1), 2), ((3, 3), 3), ((5, 4), 4), ((6, 10), 5)]
Robot 1 to_loc: (3, 8), next 3 locations: [((3, 8), (3, 7), False), ((3, 7), (3, 8), True), ((3, 8), (3, 7), False)]
Robot 2 to_loc: (3, 2), next 3 locations: [((3, 2), (3, 3), False), ((3, 3), (3, 2), True), ((3, 2), (3, 3), False)]
Robot 3 to_loc: (4, 3), next 3 locations: [((4, 3), (5, 3), False), ((5, 3), (4, 3), True), ((4, 3), (5, 3), False)]
Robot 4 to_loc: (5, 5), next 3 locations: [((5, 5), (5, 6), False), ((5, 6), (5, 5), True), ((5, 5), (5, 6), False)]
Robot 5 to_loc: (5, 10), next 3 lo

[10, 10, 10, 10, 10]

Time 11
-----------
Robot 1 is at (3, 5), intended to take the item 192, currently taking None
Robot 2 is at (3, 4), intended to take the item 195, currently taking None
Robot 3 is at (6, 4), intended to take the item 205, currently taking None
Robot 4 is at (5, 8), intended to take the item 148, currently taking None
Robot 5 is at (2, 10), intended to take the item 221, currently taking None


Robot locations: [((3, 5), 1), ((3, 4), 2), ((6, 4), 3), ((5, 8), 4), ((2, 10), 5)]
Robot 1 to_loc: (3, 6), next 3 locations: [((3, 6), (3, 5), False), ((3, 5), (3, 4), False), ((3, 4), (3, 5), True)]
Robot 2 to_loc: (2, 4), next 3 locations: [((2, 4), (3, 4), True), ((3, 4), (2, 4), False), ((2, 4), (2, 4), False)]
Robot 3 to_loc: (6, 3), next 3 locations: [((6, 3), (6, 4), False), ((6, 4), (6, 5), False), ((6, 5), (6, 4), True)]
Robot 4 to_loc: (5, 7), next 3 locations: [((5, 7), (5, 8), False), ((5, 8), (5, 9), False), ((5, 9), (5, 8), True)]
Robot 5 to_loc: (3, 10), nex

[18, 18, 18, 18, 18]

Time 19
-----------
Robot 1 is at (3, 5), intended to take the item 192, currently taking None
Robot 2 is at (2, 5), intended to take the item 195, currently taking None
Robot 3 is at (7, 5), intended to take the item None, currently taking 205
Robot 4 is at (5, 10), intended to take the item 148, currently taking None
Robot 5 is at (1, 9), intended to take the item 221, currently taking None


Robot locations: [((3, 5), 1), ((2, 5), 2), ((7, 5), 3), ((5, 10), 4), ((1, 9), 5)]
Robot 1 to_loc: (3, 4), next 3 locations: [((3, 4), (3, 3), False), ((3, 3), (3, 4), True), ((3, 4), (3, 3), False)]
Robot 2 to_loc: (2, 4), next 3 locations: [((2, 4), (2, 5), False), ((2, 5), (2, 5), False), ((2, 5), (2, 5), False)]
Robot 3 to_loc: (7, 6), next 3 locations: [((7, 6), (7, 5), True), ((7, 5), (7, 6), False), ((7, 6), (6, 6), False)]
Robot 4 to_loc: (5, 11), next 3 locations: [((5, 11), (5, 10), True), ((5, 10), (5, 11), False), ((5, 11), (5, 12), False)]
Robot 5 to_loc: (1, 

[26, 26, 26, 26, 26]

Time 27
-----------
Robot 1 is at (3, 1), intended to take the item 192, currently taking None
Robot 2 is at (2, 4), intended to take the item 195, currently taking None
Robot 3 is at (6, 6), intended to take the item None, currently taking 205
Robot 4 is at (5, 12), intended to take the item 148, currently taking None
Robot 5 is at (1, 7), intended to take the item 221, currently taking None


Robot locations: [((3, 1), 1), ((2, 4), 2), ((6, 6), 3), ((5, 12), 4), ((1, 7), 5)]
Robot 1 to_loc: (3, 2), next 3 locations: [((3, 2), (3, 1), False)]
Robot 2 to_loc: (3, 4), next 3 locations: [((3, 4), (2, 4), True), ((2, 4), (3, 4), False), ((3, 4), (3, 5), False)]
Robot 3 to_loc: (6, 5), next 3 locations: ['check', ((6, 5), (5, 5), False), ((5, 5), (6, 5), True)]
Robot 4 to_loc: (4, 12), next 3 locations: [((4, 12), (3, 12), False), ((3, 12), (4, 12), True), ((4, 12), (3, 12), False)]
Robot 5 to_loc: (1, 6), next 3 locations: [((1, 6), (1, 5), False), ((1, 5), (1, 6), T

...

[1286, 1286, 1286, 1286, 1286]

Time 1287
-----------
Robot 1 is at (3, 7), intended to take the item None, currently taking 202
Robot 2 is at (7, 14), intended to take the item None, currently taking None
Robot 3 is at (6, 0), intended to take the item None, currently taking None
Robot 4 is at (0, 14), intended to take the item None, currently taking None
Robot 5 is at (2, 6), intended to take the item None, currently taking None


Robot locations: [((3, 7), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 6), 5)]
Robot 1 to_loc: (4, 7), next 3 locations: ['check', ((4, 7), (4, 8), False), ((4, 8), (4, 7), True)]
Fictitious move for robot 2 applied
Robot 2 next 3 locations: []
Fictitious move for robot 3 applied
Robot 3 next 3 locations: []
Fictitious move for robot 4 applied
Robot 4 next 3 locations: []
Robot 5 to_loc: (2, 5), next 3 locations: [((2, 5), (2, 4), False), ((2, 4), (2, 5), True), ((2, 5), (2, 4), False)]
[((4, 7), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 5), 5

Robot 5 to_loc: (2, 3), next 3 locations: [((2, 3), (2, 2), False), ((2, 2), (2, 1), False), ((2, 1), (2, 2), True)]
[((3, 8), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 3), 5)]
[1295, 1295, 1295, 1295, 1295]

Time 1296
-----------
Robot 1 is at (3, 8), intended to take the item None, currently taking 202
Robot 2 is at (7, 14), intended to take the item None, currently taking None
Robot 3 is at (6, 0), intended to take the item None, currently taking None
Robot 4 is at (0, 14), intended to take the item None, currently taking None
Robot 5 is at (2, 3), intended to take the item None, currently taking None


Robot locations: [((3, 8), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 3), 5)]
Robot 1 to_loc: (2, 8), next 3 locations: [((2, 8), (2, 7), False), ((2, 7), (2, 8), True), ((2, 8), (2, 7), False)]
Fictitious move for robot 2 applied
Robot 2 next 3 locations: []
Fictitious move for robot 3 applied
Robot 3 next 3 locations: []
Fictitious move for robot 4 applied
Robot 4 ne

[1312, 1312, 1312, 1312, 1312]

Time 1313
-----------
Robot 1 is at (1, 7), intended to take the item None, currently taking 202
Robot 2 is at (7, 14), intended to take the item None, currently taking None
Robot 3 is at (6, 0), intended to take the item None, currently taking None
Robot 4 is at (0, 14), intended to take the item None, currently taking None
Robot 5 is at (2, 0), intended to take the item None, currently taking None


Robot locations: [((1, 7), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
Robot 1 to_loc: (1, 8), next 3 locations: [((1, 8), (1, 7), False), ((1, 7), (2, 7), False), ((2, 7), (1, 7), True)]
Fictitious move for robot 2 applied
Robot 2 next 3 locations: []
Fictitious move for robot 3 applied
Robot 3 next 3 locations: []
Fictitious move for robot 4 applied
Robot 4 next 3 locations: []
Fictitious move for robot 5 applied
Robot 5 next 3 locations: []
[((1, 8), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
[1313, 1313, 1313, 1313, 1313]

T

[((1, 8), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
[1321, 1321, 1321, 1321, 1321]

Time 1322
-----------
Robot 1 is at (1, 8), intended to take the item None, currently taking 202
Robot 2 is at (7, 14), intended to take the item None, currently taking None
Robot 3 is at (6, 0), intended to take the item None, currently taking None
Robot 4 is at (0, 14), intended to take the item None, currently taking None
Robot 5 is at (2, 0), intended to take the item None, currently taking None


Robot locations: [((1, 8), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
Robot 1 to_loc: (2, 8), next 3 locations: [((2, 8), (1, 8), False), ((1, 8), (0, 8), False), ((0, 8), (1, 8), True)]
Fictitious move for robot 2 applied
Robot 2 next 3 locations: []
Fictitious move for robot 3 applied
Robot 3 next 3 locations: []
Fictitious move for robot 4 applied
Robot 4 next 3 locations: []
Fictitious move for robot 5 applied
Robot 5 next 3 locations: []
[((2, 8), 1), ((7, 14), 2), ((6, 

Robot 4 next 3 locations: []
Fictitious move for robot 5 applied
Robot 5 next 3 locations: []
[((1, 7), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
[1330, 1330, 1330, 1330, 1330]

Time 1331
-----------
Robot 1 is at (1, 7), intended to take the item None, currently taking 202
Robot 2 is at (7, 14), intended to take the item None, currently taking None
Robot 3 is at (6, 0), intended to take the item None, currently taking None
Robot 4 is at (0, 14), intended to take the item None, currently taking None
Robot 5 is at (2, 0), intended to take the item None, currently taking None


Robot locations: [((1, 7), 1), ((7, 14), 2), ((6, 0), 3), ((0, 14), 4), ((2, 0), 5)]
Robot 1 to_loc: (0, 7), next 3 locations: [((0, 7), (1, 7), False)]
Fictitious move for robot 2 applied
Robot 2 next 3 locations: []
Fictitious move for robot 3 applied
Robot 3 next 3 locations: []
Fictitious move for robot 4 applied
Robot 4 next 3 locations: []
Fictitious move for robot 5 applied
Robot 5 next 3 lo

example = Warehouse('wh3.p','items_list.p')

example = main_program(example)